In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"
cohort = "GSE115574"

# Input paths
in_trait_dir = "../../input/GEO/Arrhythmia"
in_cohort_dir = "../../input/GEO/Arrhythmia/GSE115574"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/GSE115574.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/GSE115574.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/GSE115574.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression data from human left and right atrial tissues in patients with degenerative MR in SR and AFib."
!Series_summary	"We aimed to compare the gene expression profiles of patients with degenerative MR in SR and AFib. We used Affymetrix human gene expression microarrays for each atrium sample. We chose most homogenous groups to compare, cause of the noise overshadows in high-throughput analysis when investigating complex diseases."
!Series_overall_design	"Left and right atrial tissue samples were obtained from patients with chronic primary severe MR in permanent AFib (n=15) and sinus rhythm (n=15). Transcriptomic analysis and bioinformatics have been done on all atrial tissues. Independent datasets from GEO were included in the analysis to confirm our findings. Real-time qPCR used to validate microarray results. Atrial tissues investigated via transmission electron microscopy for ultrasutructural changes."
Sample Characteristics Dictionar

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Evaluate gene expression data availability 
# Based on background information, this dataset contains gene expression data from human atrial tissues
is_gene_available = True

# 2. Identify variable availability and create data type conversion functions

# 2.1 & 2.2 For trait (Arrhythmia - atrial fibrillation)
# From sample characteristics, we see the trait key is 0, with values related to atrial fibrillation vs sinus rhythm
trait_row = 0

def convert_trait(value):
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary: 1 for atrial fibrillation, 0 for sinus rhythm
    if 'atrial fibrillation' in value.lower():
        return 1
    elif 'sinus rhythm' in value.lower():
        return 0
    else:
        return None

# 2.1 & 2.2 For age
# Age information is not provided in the sample characteristics
age_row = None

def convert_age(value):
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Convert to float for continuous variable
        return float(value)
    except:
        return None

# 2.1 & 2.2 For gender
# Gender information is not provided in the sample characteristics
gender_row = None

def convert_gender(value):
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    # Convert to binary: 0 for female, 1 for male
    if value in ['female', 'f']:
        return 0
    elif value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Determine trait data availability and save metadata
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Create clinical data DataFrame from the sample characteristics dictionary
    # The sample characteristics dictionary is from the previous step:
    # {0: ['disease state: atrial fibrillation patient with severe mitral regurgitation', 
    #      'disease state: sinus rhythm patient with severe mitral regurgitation'], 
    #  1: ['tissue: left atrium  - heart', 'tissue: right atrium  - heart']}
    
    # First, create a DataFrame with the provided characteristics
    sample_chars = {
        0: ['disease state: atrial fibrillation patient with severe mitral regurgitation', 
            'disease state: sinus rhythm patient with severe mitral regurgitation'],
        1: ['tissue: left atrium  - heart', 'tissue: right atrium  - heart']
    }
    
    # Create a dictionary to represent the clinical data
    clinical_dict = {}
    for key, values in sample_chars.items():
        clinical_dict[key] = values
    
    # Create DataFrame from the dictionary
    clinical_data = pd.DataFrame(clinical_dict)
    
    # Select clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical dataframe to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview:
{0: [1.0], 1: [nan]}
Clinical data saved to ../../output/preprocess/Arrhythmia/clinical_data/GSE115574.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Arrhythmia/GSE115574/GSE115574_series_matrix.txt.gz


Gene data shape: (54675, 59)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Observe gene identifiers from the previous step output
# These identifiers appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at')
# rather than standard human gene symbols (which would be like BRCA1, TP53, etc.)
# Affymetrix probe IDs typically need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

Column 'ID' contains values matching gene data ID pattern


Column 'GB_ACC' contains values matching gene data ID pattern


Column 'Target Description' contains values matching gene data ID pattern


Column 'Representative Public ID' contains values matching gene data ID pattern


...

Column 'Gene Symbol' contains values matching gene data ID pattern


Column 'ENTREZ_GENE_ID' contains values matching gene data ID pattern


Column 'RefSeq Transcript ID' contains values matching gene data ID pattern


Column 'Gene Ontology Biological Process' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Species Scientific Name' may contain gene-related information
Sample values: ['Homo sapiens', 'Homo sapiens', 'Homo sapiens']
Column 'Target Description' may contain gene-related information
Sample values: ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene"]
Column 'Gene Title' may contain gene-related information
Sample values: ['discoidin domain receptor tyrosine kinase 1 /// microRNA 4640', 'replication factor C (activator 1) 2, 40kDa', "heat shock 70kDa protein 6 (HSP70B')"]
Column 'Gene Symbol' may contain gene-related information
Sample values: ['DDR1 /// MIR4640', 'RFC2', 'HSPA6']
Column 'ENTREZ_GENE_ID' may contain gene-related information
Sample values: ['780 /// 100616237

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation dataframe contain the gene IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column contains the probe identifiers that match the gene expression data (e.g., '1007_s_at')
# - 'Gene Symbol' column contains the gene symbols (e.g., 'DDR1 /// MIR4640')

# 2. Get gene mapping dataframe by extracting the ID and Gene Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("First few genes after mapping:")
print(gene_data.index[:10].tolist())

# 4. Normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression dataframe shape after normalization: {gene_data.shape}")
print("First few genes after normalization:")
print(gene_data.index[:10].tolist())

# Save the gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}


Gene expression dataframe shape after mapping: (21278, 59)
First few genes after mapping:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
Gene expression dataframe shape after normalization: (19845, 59)
First few genes after normalization:
['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']


Gene data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE115574.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (19845, 59)
Gene data shape after normalization: (19845, 59)


Normalized gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE115574.csv
Original clinical data preview:
         !Sample_geo_accession  \
0  !Sample_characteristics_ch1   
1  !Sample_characteristics_ch1   

                                          GSM3182680  \
0  disease state: atrial fibrillation patient wit...   
1                       tissue: left atrium  - heart   

                                          GSM3182681  \
0  disease state: atrial fibrillation patient wit...   
1                      tissue: right atrium  - heart   

                                          GSM3182682  \
0  disease state: atrial fibrillation patient wit...   
1                       tissue: left atrium  - heart   

                                          GSM3182683  \
0  disease state: atrial fibrillation patient wit...   
1                      tissue: right atrium  - heart   

                                          GSM3182684  \
0  disease state: atrial fibrilla

Linked data shape before processing: (59, 19846)
Linked data preview (first 5 rows, 5 columns):
            Arrhythmia      A1BG  A1BG-AS1      A1CF        A2M
GSM3182680         1.0  4.926930  3.103542  5.365683  13.884166
GSM3182681         1.0  4.932586  3.398962  5.590272  13.669718
GSM3182682         1.0  4.606200  3.103393  5.579943  13.626048
GSM3182683         1.0  5.171341  3.291102  5.255341  13.419765
GSM3182684         1.0  4.944955  3.197841  5.753891  13.931109


Data shape after handling missing values: (59, 19846)
For the feature 'Arrhythmia', the least common label is '1.0' with 28 occurrences. This represents 47.46% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

Data shape after removing biased features: (59, 19846)
A new JSON file was created at: ../../output/preprocess/Arrhythmia/cohort_info.json


Linked data saved to ../../output/preprocess/Arrhythmia/GSE115574.csv
